In [ ]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

In [ ]:
import sys
import os

path2cpp_pkg = "/Users/mariusmahiout/Documents/repos/ising_core/build"
sys.path.append(path2cpp_pkg)

import ising
import os
import sys
import time

import preprocessing as pre
import model_eval as eval
import utils as utils
import misc_plotting as misc_plotting
import isingfitter as fitter

os.chdir("../..")
os.getcwd()

import numpy as np
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import pandas as pd
import tqdm

import numpy as np
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from IPython.display import display, HTML
from ipywidgets import HBox, VBox, widgets
import scipy
import pandas as pd

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
import seaborn as sns
sns.set_theme()
sns.set_style(rc = {'axes.facecolor': '#e5ecf6'})
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

In [ ]:
data_folder = "./data/"
mouse_name = "Angie"

def load_params(num_units: int, data_folder: str, mouse_name: str):
    return np.load(data_folder + "params/" + f'params_{mouse_name}_{num_units}.npy', allow_pickle=True).item()

num_units_range = range(10, 270 + 10, 10)

params = dict(
    (num_units, load_params(num_units, data_folder, mouse_name)) for num_units in num_units_range
)

# EQ params
eq_couplings = dict(
    (num_units, params[num_units]["eq_couplings"]) for num_units in num_units_range
)
eq_fields = dict(
    (num_units, params[num_units]["eq_fields"]) for num_units in num_units_range
)

# NEQ params
neq_couplings = dict(
    (num_units, params[num_units]["neq_couplings"]) for num_units in num_units_range
)
neq_fields = dict(
    (num_units, params[num_units]["neq_fields"]) for num_units in num_units_range
)

num_units = 50
list(num_units_range)

In [ ]:
bin_width = 50 # ms

mouse_name = "Angie"
# angie performing 1
recording_fname = "RESULTS_Angie_20170825_1220_allbeh_1000s.mat"

sample = pre.get_recording_sample(
    fname=recording_fname, 
    mouse_name=mouse_name, 
    bin_width=bin_width, 
    num_units=num_units
)

In [ ]:
J_eq = eq_couplings[num_units]
h_eq = eq_fields[num_units]

J_neq = neq_couplings[num_units]
h_neq = neq_fields[num_units]

In [ ]:
##############
# LIKELIHOOD #
##############

num_sims = 150_000
num_burn = 5000
lr = 0.1
win_size = 10
tol_ml = 1e-4
max_steps = 1


eq_model = ising.EqModel(J_eq, h_eq)

eq_fitter = fitter.EqFitter(eq_model)

In [ ]:
# # inference
# use_llh = False
# eq_fitter.maximize_likelihood(
#     sample=sample,
#     max_steps=2000, 
#     learning_rate=0.0005,
#     win_size = win_size,
#     tolerance= tol_ml, 
#     num_sims=num_sims, 
#     num_burn=num_burn,
#     calc_llh=use_llh
# )

In [ ]:
##############
# LIKELIHOOD #
##############

neq_model = ising.NeqModel(J_neq, h_neq)

neq_fitter = fitter.NeqFitter(neq_model)

In [ ]:
# # inference
# use_llh = False
# neq_fitter.maximize_likelihood(
#     sample=sample, 
#     max_steps=2000, 
#     learning_rate=0.001,
#     win_size = win_size,
#     tolerance= tol_ml, 
#     calc_llh=use_llh
# )

In [ ]:
# misc_plotting.convergence_plot(neq_fitter, plot_llh=use_llh, path=f"./analyses/neq_convergence/rec_neq{num_units}.pdf")

In [ ]:
eq_sample = eq_model.simulate(num_sims, num_burn)
neq_sample = neq_model.simulate(num_sims, num_burn)

In [ ]:
labels = ["EQ", "NEQ"]
metadata = utils.get_metadata(
    num_units = num_units,
    is_empirical_analysis = True,
    bin_width = bin_width,
    num_bins = sample.getNumBins(),
    eq_inv_methods = ["ML"],
    neq_inv_methods=["ML"],
    num_steps="...",
    learning_rate="...",
    is_converged="...",
    num_sims_ml=num_sims,
    num_burn_ml=num_burn,
)


analysis_name = "recording_analysis"
bin_width = 0
analysis_path = utils.get_analysis_path(analysis_name, num_units, bin_width)


layout_spec = {
    ("means", "scatter"): (1, 1),
    ("pcorrs", "scatter"): (1, 2),
    ("dcorrs", "scatter"): (1, 3),
    #("num-distr", "density"): 2,
}

ising_eval = eval.IsingEval(
    analysis_path=analysis_path,
    metadata=metadata,
    true_model=None,
    est_models=[eq_model, neq_model],
    true_sample=sample,
    est_samples=[eq_sample, neq_sample],
    labels=labels,
    layout_spec=layout_spec, 
    is_eq_vs_neq=True
)
ising_eval.generate_plots(is_pdf=True)

In [ ]:
def get_av_magnetization(states):
    N, num_bins = states.shape
    m_av = np.sum(np.abs(np.sum(states, axis=0)) / N)
    return m_av / num_bins

def get_av_susceptibility(states, T):
    N = states.shape[0]
    M = np.sum(states, axis=0) # array of length num_bins
    M_av = np.mean(M)
    M_sq_av = np.mean(M ** 2)
    return (M_sq_av - M_av ** 2) / (T * N)

num_units = eq_model.getNumUnits()

J, h = eq_model.getCouplings(), eq_model.getFields()

num_betas = 500
beta_range = np.linspace(0, 2, num_betas)
av_mag = dict(
    (beta, None) for beta in beta_range
)
av_sus = dict(
    (beta, None) for beta in beta_range
)

for beta in beta_range:
    model = ising.EqModel(J * beta, h * beta)
    sim = model.simulate(15_000, 5_000)

    av_mag[beta] = get_av_magnetization(sim.getStates())
    av_sus[beta] = get_av_susceptibility(sim.getStates(), T=1/beta)


In [ ]:
# plotting
fig, ax = plt.subplots(figsize=(15,10))
ax.plot(beta_range, list(av_mag.values()), marker='o', color="darkcyan", label=r"$\langle |M| \rangle$")
ax.plot(beta_range, list(av_sus.values()), marker='d', color="darkorange", label=r"$\langle \chi \rangle$")

ax.set_xlabel(r"$\beta$", fontsize=22)
ax.legend(fontsize=22)
plt.ylim([0,1.25])

plt.savefig(f"neural_phase_transition_{num_units}.pdf")

plt.show()